In [1]:
import pandas as pd
import pymysql.cursors

In [2]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ml', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = "select issue,matchid from `breadt_lottery_info` where time > '2018-01-01';"
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    lottery_df = pd.DataFrame(rows)
    
    connection.close()

In [3]:
result_df = pd.read_csv('../data/submission_2019.csv')

In [4]:
df = pd.merge(lottery_df, result_df, on='matchid', how='left')
df = df.dropna()
df = df.drop_duplicates()

In [5]:
df['score'] = df.apply(lambda row: 1 if row.result == row.pred else 0, axis=1)
df['exist'] = 1

In [6]:
result_df[result_df['matchid'] == 2405531]

,matchid,result,pred
1343,2405531,2,2.0


In [7]:
df[df['issue'] == 19061]

,issue,matchid,result,pred,score,exist
3223,19061,2411095,0.0,1.0,0,1
3224,19061,2411100,1.0,1.0,1,1
3225,19061,2402946,2.0,2.0,1,1
3226,19061,2402948,1.0,2.0,0,1
3227,19061,2529128,2.0,2.0,1,1
3228,19061,2529126,2.0,2.0,1,1
3229,19061,2529124,1.0,2.0,0,1
3230,19061,2529132,2.0,2.0,1,1
3231,19061,2529130,0.0,2.0,0,1
3232,19061,2529127,0.0,2.0,0,1


In [8]:
t = df
final_result = t.groupby(t['issue'])[['score','exist']].sum()

In [9]:
f = final_result[final_result['exist'] > 0]
f.sort_values(by=['score', 'exist'], ascending=False)

,score,exist
issue,,
19055,14,14
19032,13,14
19051,13,14
19031,12,14
19043,12,14
19047,12,14
19052,12,14
19058,12,14
19028,11,14


+ 19038 - 19041 都是国家队比赛 应该是世界杯
+ 19030 19034 全部是亚赛和欧赛
